# Train YOLO Model in Google Colab

**GitHub:** [Train YOLO Model](https://github.com/Etaizil/Train_Yolo_Model)


**Verify NVIDIA GPU Availability**

Make sure you're using a GPU-equipped machine by going to "Runtime" -> "Change runtime type" in the top menu bar, and then selecting one of the GPU options in the Hardware accelerator section. Click Play on the following code block to verify that the NVIDIA GPU is present and ready for training.

In [ ]:
!nvidia-smi

# Imports

In [ ]:
from pathlib import Path
import os
import sys
import random
import shutil

# Upload Image Dataset and Prepare Training Data

Next, we'll upload our dataset and prepare it for training with YOLO. We'll split the dataset into train and validation folders, and we'll automatically generate the configuration file for training the model.

## Upload images

First, we need to upload the dataset to Colab. Here are a few options for moving the `data.zip` folder into this Colab instance.

In [ ]:
!gdown --id ID_OF_DATA -O /content/data.zip

## Split images into train and validation folders

 Click the folder icon on the left and see your `data.zip` file in the list of files.
 
 Next, we'll unzip `data.zip` and create some folders to hold the images.
 
 Run the following code block to unzip the data.

In [5]:
# Unzip images to a custom data folder
!unzip -q /content/data.zip -d /content/custom_data

Ultralytics requires a particular folder structure to store training data for models. The root folder is named “data”. Inside, there are two main folders:

*   **Train**: These are the actual images used to train the model. In one epoch of training, every image in the train set is passed into the neural network. The training algorithm adjusts the network weights to fit the data in the images.


*   **Validation**: These images are used to check the model's performance at the end of each training epoch.

In [6]:
data_path = "/content/custom_data"
train_percent = 0.8

if not os.path.isdir(data_path):
    print(f'Directory {data_path} not found. Verify the path and try again.')
    sys.exit(0)

if train_percent < 0.01 or train_percent > 0.99:
    print('Invalid entry for train_pct. Please enter a number between 0.01 and 0.99.')
    sys.exit(0)

val_percent = 1 - train_percent

input_image_path = os.path.join(data_path, 'images')
input_label_path = os.path.join(data_path, 'labels')

cwd = os.getcwd()
train_img_path = os.path.join(cwd, 'data/train/images')
train_txt_path = os.path.join(cwd, 'data/train/labels')
val_img_path = os.path.join(cwd, 'data/validation/images')
val_txt_path = os.path.join(cwd, 'data/validation/labels')

for dir_path in [train_img_path, train_txt_path, val_img_path, val_txt_path]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print(f'Created folder at {dir_path}.')

img_file_list = list(Path(input_image_path).rglob('*'))
txt_file_list = list(Path(input_label_path).rglob('*'))

print(f'Number of image files: {len(img_file_list)}')
print(f'Number of annotation files: {len(txt_file_list)}')

file_num = len(img_file_list)
train_num = int(file_num * train_percent)
val_num = file_num - train_num
print(f'Images moving to train: {train_num}')
print(f'Images moving to validation: {val_num}')

random.shuffle(img_file_list)

for i, set_num in enumerate([train_num, val_num]):
    for _ in range(set_num):
        if not img_file_list:
            break

        img_path = img_file_list.pop()
        img_fn = img_path.name
        base_fn = img_path.stem
        txt_fn = base_fn + '.txt'
        txt_path = os.path.join(input_label_path, txt_fn)

        if i == 0:
            new_img_path, new_txt_path = train_img_path, train_txt_path
        else:
            new_img_path, new_txt_path = val_img_path, val_txt_path

        shutil.copy(img_path, os.path.join(new_img_path, img_fn))

        if os.path.exists(txt_path):
            shutil.copy(txt_path, os.path.join(new_txt_path, txt_fn))


Created folder at /content/data/train/images.
Created folder at /content/data/train/labels.
Created folder at /content/data/validation/images.
Created folder at /content/data/validation/labels.
Number of image files: 1187
Number of annotation files: 1187
Images moving to train: 949
Images moving to validation: 238


# Install Requirements (Ultralytics)

Next, we'll install the Ultralytics library in this Google Colab instance. This Python library will be used to train the YOLO model.

In [ ]:
!pip install ultralytics

# Configure Training


There's one last step before we can run training: we need to create the Ultralytics training configuration YAML file. This file specifies the location of your train and validation data, and it also defines the model's classes.

Run the code block below to generate a `data.yaml` configuration file. Make sure you have a labelmap file located at `custom_data/classes.txt`. If you used Label Studio, it should already be present. If you assembled the dataset another way, you may have to manually create the `classes.txt` file.

In [8]:
import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

In [ ]:
path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

Created config file at /content/data.yaml

File contents:

path: /content/data
train: train/images
val: validation/images
nc: 3
names:
- Happy
- Natural
- Surprised


# Training

In [ ]:
!yolo detect train data=/content/data.yaml model=yolo11m.pt epochs=30 imgsz=640

Ultralytics 8.3.88 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/content/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

# Test Model

In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

Ultralytics 8.3.88 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,032,345 parameters, 0 gradients, 67.7 GFLOPs

image 1/238 /content/data/validation/images/01e0ccd3-WIN_20250307_22_07_20_Pro_2.jpg: 384x640 1 Natural, 72.0ms
image 2/238 /content/data/validation/images/0226374b-WIN_20250307_22_05_21_Pro.jpg: 384x640 1 Natural, 24.1ms
image 3/238 /content/data/validation/images/0240ae34-WIN_20250307_22_25_25_Pro.jpg: 384x640 1 Surprised, 24.0ms
image 4/238 /content/data/validation/images/038c453c-WIN_20250307_22_06_06_Pro.jpg: 384x640 1 Natural, 24.0ms
image 5/238 /content/data/validation/images/03e8adee-WIN_20250307_22_12_31_Pro.jpg: 384x640 1 Happy, 24.0ms
image 6/238 /content/data/validation/images/052bcc02-WIN_20250307_22_29_22_Pro.jpg: 384x640 1 Surprised, 20.1ms
image 7/238 /content/data/validation/images/056fb0f4-WIN_20250307_22_11_19_Pro_2.jpg: 384x640 1 Happy, 20.1ms
image 8/238 /content/data/validation/images/060230d9-WIN_2

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:5]:
  display(Image(filename=image_path, height=400))
  print('\n')


# Download Model

In [ ]:
# Create "my_model" folder to store model weights and train results
!mkdir /content/my_model
!cp /content/runs/detect/train/weights/best.pt /content/my_model/my_model.pt
!cp -r /content/runs/detect/train /content/my_model

%cd my_model
!zip /content/my_model.zip my_model.pt
!zip -r /content/my_model.zip train
%cd /content

/content/my_model
  adding: my_model.pt (deflated 38%)
  adding: train/ (stored 0%)
  adding: train/results.csv (deflated 59%)
  adding: train/labels_correlogram.jpg (deflated 44%)
  adding: train/args.yaml (deflated 53%)
  adding: train/weights/ (stored 0%)
  adding: train/weights/last.pt (deflated 38%)
  adding: train/weights/best.pt (deflated 38%)
  adding: train/labels.jpg (deflated 39%)
  adding: train/events.out.tfevents.1741785176.cb4357930276.777.0 (deflated 93%)
  adding: train/train_batch1.jpg (deflated 7%)
  adding: train/train_batch2.jpg (deflated 8%)
  adding: train/train_batch0.jpg (deflated 10%)
/content


In [ ]:
from google.colab import files

files.download('/content/my_model.zip')

## Deploy YOLO Model on PC

Next, we'll take our downloaded model and run it on a local device. This section provides instructions showing how to deploy YOLO models on various devices.

The easiest way to run Ultralytics YOLO models on a PC is using **Anaconda**, which helps set up a virtual Python environment with all necessary dependencies.

### **1. Download and Install Anaconda**
[Download Anaconda](https://anaconda.com/download) and follow the default installation settings.

### **2. Set up a virtual environment**
Open **Anaconda Prompt** (or a terminal on macOS/Linux) and create a new Python environment:
```
conda create --name yolo-env python=3.12 -y
conda activate yolo-env
```

Then, install Ultralytics:

```
pip install ultralytics
```

For NVIDIA GPU users, install the GPU-accelerated version of PyTorch:

```
pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
```

### 3. Extract the trained model
Unzip ```my_model.zip``` and navigate into its folder:
```
cd path/to/folder
```
#### 3.1. Download my model
If you want to, you may download my face expressions trained model and run it.

Simply click ```my_model.pt``` and select 'Download raw file'.

### 4. Download and run yolo_detect.py
Download the detection script:

```
curl -o yolo_detect.py https://raw.githubusercontent.com/Etaizil/Train_Yolo_Model/refs/heads/main/yolo_detect.py
```

Run inference with a YOLOv8n model on a laptop or USB camera at 1280x720 resolution:

```
python yolo_detect.py --model my_model.pt --source laptop0 --resolution 720x720
```

This will open a window displaying a live feed with bounding boxes drawn around detected facial expressions.

Alternatively, to process an image, video, or folder of images, specify the source:

```
python yolo_detect.py --model my_model.pt --source path/to/image_or_video.mp4
```

You can also run the model on an video file. For full instructions see the [README file](https://github.com/Etaizil/Train_Yolo_Model/blob/main/README.md).


